## Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob
import random
import cv2
import sys
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from MAD_SVM import classify_img
from sklearn.preprocessing import StandardScaler

## Read images 

In [ ]:
# folders =["glioma","meningioma","notumor","pituitary"]
glioma = []
meningioma =[]
notumor = []
pituitary=[]

for f in glob.iglob(r"C:\Users\esraa omaar\Documents\college_materail\3rdYear_term2\Advanced_ML\Training\glioma\*.jpg"):
    img = cv2.imread(f)
    img= cv2.resize(img,(128,128))
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    glioma.append(img)
for f in glob.iglob(r"C:\Users\esraa omaar\Documents\college_materail\3rdYear_term2\Advanced_ML\Training\meningioma\*.jpg"):
    img = cv2.imread(f)
    img= cv2.resize(img,(128,128))
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    meningioma.append(img)
    
# no tumor
for f in glob.iglob(r"C:\Users\esraa omaar\Documents\college_materail\3rdYear_term2\Advanced_ML\Training\notumor\*.jpg"):
    img = cv2.imread(f)
    img= cv2.resize(img,(128,128))
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    notumor.append(img)
    
# pituitary
for f in glob.iglob(r"C:\Users\esraa omaar\Documents\college_materail\3rdYear_term2\Advanced_ML\Training\pituitary\*.jpg"):
    img = cv2.imread(f)
    img= cv2.resize(img,(128,128))
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    pituitary.append(img)
        


In [ ]:
glioma = np.array(glioma)
meningioma =np.array(meningioma)
notumor = np.array(notumor)
pituitary=np.array(pituitary)

# Visualize tumor brain image

In [ ]:
# vis random images from labels
gl_img = glioma[np.random.choice(glioma.shape[0],5,replace= False)]
plt.figure(figsize =(16,9))

for i in range(5):
    plt.subplot(1,5,i+1)
    plt.title('glioma')
    plt.imshow(gl_img[i])

In [ ]:
men_img = meningioma[np.random.choice(meningioma.shape[0],5,replace= False)]
plt.figure(figsize =(16,9))

for i in range(5):
    plt.subplot(1,5,i+1)
    plt.title('meningioma')
    plt.imshow(men_img[i])

In [ ]:
no_img = notumor[np.random.choice(notumor.shape[0],5,replace= False)]
plt.figure(figsize =(16,9))

for i in range(5):
    plt.subplot(1,5,i+1)
    plt.title('notumor')
    plt.imshow(no_img[i])

In [ ]:
pit_img = pituitary[np.random.choice(pituitary.shape[0],5,replace= False)]
plt.figure(figsize =(16,9))

for i in range(5):
    plt.subplot(1,5,i+1)
    plt.title('pituitary')
    plt.imshow(pit_img[i])

## Image preprocessing

In [ ]:
# Function to load images from a directory
def load_images_from_dir(directory):
    images = []
    labels = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg'):
                filepath = os.path.join(root, file)
                label = os.path.basename(os.path.dirname(filepath))
                labels.append(label)
                image = cv2.imread(filepath)
                image = cv2.resize(image, (128, 128))
                images.append(image)
    return np.array(images), np.array(labels)

In [ ]:
def norm(X, X_test):
    # Normalize
    X = X.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    return X, X_test

def encode(y, y_test):
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_test_encoded = label_encoder.transform(y_test)
    print('normalization and encoding done.')
    
    return y_encoded, y_test_encoded

In [ ]:
def apply_pca(X, X_test):
    pca = PCA(n_components=128) 
    X_pca = pca.fit_transform(X.reshape(X.shape[0], -1))
    X_test_pca = pca.transform(X_test.reshape(X_test.shape[0], -1))

    print('X.shape:', X.shape)
    print('X_pca.shape: ', X_pca.shape)
    
    return X_pca, X_test_pca


In [ ]:
def create_model(X, y): 
    
    # Define the model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X.shape[1],)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        # Dense(32, activation='relu'),
        Dense(len(np.unique(y)), activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    print('Model created.')
    
    return model

In [ ]:
# Define training and testing directories
train_dir = 'Training'
test_dir = 'Testing'

# Load images and labels
X, y = load_images_from_dir(train_dir)
X_test, y_test = load_images_from_dir(test_dir)

In [ ]:
X , X_test = norm(X, X_test)
X , X_test = apply_pca(X, X_test)


y, y_test = encode(y, y_test)


In [ ]:
model = create_model(X, y)

# Train the model on the entire training dataset
history = model.fit(X, y, epochs=25, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'Test accuracy: {test_acc}')

In [ ]:
model.summary()

In [ ]:
def plot_confusion(model, X_test, y_test):
    # Define the class labels
    labels = ['glioma', 'meningioma', 'notumor', 'pituitary']
    # Get raw predictions for the test data
    y_pred_probs = model.predict(X_test)

    # Convert probabilities to class labels
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Create the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Plot the confusion matrix
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
plot_confusion(model, X_test, y_test)

In [ ]:
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']


index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]

Epochs = [i + 1 for i in range(len(tr_acc))]
loss_label = f'Best epoch = {str(index_loss + 1)}'
acc_label = f'Best epoch = {str(index_acc + 1)}'


plt.figure(figsize=(20, 12))
plt.style.use('fivethirtyeight')


plt.subplot(2, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label='Training loss')
plt.plot(Epochs, val_loss, 'g', label='Validation loss')
plt.scatter(index_loss + 1, val_lowest, s=150, c='blue', label=loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(2, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label='Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label='Validation Accuracy')
plt.scatter(index_acc + 1, acc_highest, s=150, c='blue', label=acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.suptitle('Model Training Metrics Over Epochs', fontsize=16)
plt.show()

In [ ]:
def run():
    # Define training and testing directories
    train_dir = 'Training'
    test_dir = 'Testing'

    # Load images and labels
    X, y = load_images_from_dir(train_dir)
    X_test, y_test = load_images_from_dir(test_dir)
    
    X , X_test = norm(X, X_test)
    X , X_test = apply_pca(X, X_test)
    y, y_test = encode(y, y_test)
    
    model = create_model(X, y)

    # Train the model on the entire training dataset
    history = model.fit(X, y, epochs=25, batch_size=32, validation_split=0.2, verbose=1)
    
    return model, history, X_test, y_test

In [ ]:
import tkinter as tk
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

model, history, X_test, y_test = run()

# Define your functions for plotting accuracy and confusion matrix
def plot_accuracy(history):
    # Plot accuracy
    fig = plt.Figure(figsize=(10, 4))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(history.history['accuracy'], label='Training Accuracy')
    ax.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax.set_title('Training and Validation Accuracy')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.legend()
    
    return fig

def plot_cm(model, X_test, y_test):
    # Define the class labels
    labels = ['glioma', 'meningioma', 'notumor', 'pituitary']
    # Get raw predictions for the test data
    y_pred_probs = model.predict(X_test)

    # Convert probabilities to class labels
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Create the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Plot the confusion matrix in a new window
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(1, 1, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')
    ax.set_title('Confusion Matrix')
    
    return fig

def plot_and_evaluate(graph_type):
    if graph_type == 'confusion_matrix':
        fig = plot_cm(model, X_test, y_test)
    elif graph_type == 'loss_curve':
        fig = plot_accuracy(history)
    
    # Create a Tkinter window for the plot
    plot_window = tk.Toplevel(root)
    plot_window.title(graph_type.replace('_', ' ').title())
    
    # Embed the figure in the Tkinter window
    canvas = FigureCanvasTkAgg(fig, master=plot_window)
    canvas.draw()
    canvas.get_tk_widget().pack()

# GUI
root = tk.Tk()

root.minsize(500, 200)
# root.iconbitmap('icon.jpg')

# label
label = tk.Label(text="Generate Graph")
label.config(font=('Courier', 32))
label.pack()

def plot_cm_gui():
    plot_and_evaluate('confusion_matrix')

def plot_lc_gui():
    plot_and_evaluate('loss_curve')

# buttons
cm_button = tk.Button(root, text='Confusion Matrix', command=plot_cm_gui)
cm_button.pack()

lc_button = tk.Button(root, text='Loss Curve', command=plot_lc_gui)
lc_button.pack()

root.mainloop()

In [ ]:
from tkinter import Tk, ttk, filedialog, Canvas, Label, Button
from PIL import Image, ImageTk
import cv2
import numpy as np
import joblib  # for loading SVM model
from keras.models import load_model
from MAD_SVM import classify_img  # Import classify_img function

# Load the ANN model for the ANN tab
ann_model = load_model('tumorANN2.h5')

# Load the SVM model for the SVM tab
svm_model=[]

# Dark theme colors
BACKGROUND_COLOR = "#333333"
FOREGROUND_COLOR = "#ffffff"
BUTTON_COLOR = "#555555"

class ImageTab:
    def __init__(self, master, model):
        self.model = model
        self.frame = ttk.Frame(master, style='Dark.TFrame')
        self.button = ttk.Button(self.frame, text="Upload Image", command=self.upload_image, style='Dark.TButton')
        self.button.pack()
        self.canvas = Canvas(self.frame, width=450, height=400, bg=BACKGROUND_COLOR, borderwidth=0, highlightthickness=0)
        self.canvas.pack()
        self.prediction_label = Label(self.frame, text="Predicted class: ", bg=BACKGROUND_COLOR, fg=FOREGROUND_COLOR)
        self.prediction_label.pack()

    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            image = Image.open(file_path)
            # Resize the image to fit within the canvas, preserving aspect ratio
            max_size = (450, 400)
            image.thumbnail(max_size, Image.ANTIALIAS)
            self.photo = ImageTk.PhotoImage(image)  # Keep a reference to the image object
            # Calculate the position to center the image
            x = (max_size[0] - image.width) / 2
            y = (max_size[1] - image.height) / 2
            # Display the image on the canvas
            self.canvas.create_image(x, y, image=self.photo, anchor='nw')
            
            # Classify the image using the model
            if self.model == ann_model:
                prediction = ann_model.predict(self.preprocess_image(file_path))
                class_labels = {0: 'glioma', 1: 'meningioma', 2: 'notumor', 3: 'pituitary'}
                # Get the name of the class
                predicted_class_index = np.argmax(prediction)  # Get the index of the maximum value
                predicted_class_name = class_labels[predicted_class_index]  # Get the class label corresponding to the index
                self.prediction_label.config(text="Predicted class: " + predicted_class_name)

            elif self.model == svm_model:
                prediction = classify_img(file_path)
                self.prediction_label.config(text="Predicted class: " + prediction)

    def preprocess_image(self, image_path):
        # Read the image using OpenCV
        image = cv2.imread(image_path)
        # Resize the image to 64x64 pixels
        image = cv2.resize(image, (64, 64))
        # Add an extra dimension to the image
        image = np.expand_dims(image, axis=0)
        # Convert the image to a numpy array and normalize pixel values
        image = np.array(image, dtype=np.float32) / 255.0
        return image

def create_tab(parent, text, model):
    tab = ImageTab(parent, model)
    parent.add(tab.frame, text=text)

root = Tk()
root.title("Advanced Machine Project")

# Create a custom style for dark theme
style = ttk.Style(root)
style.theme_use('clam')
style.configure('Dark.TFrame', background=BACKGROUND_COLOR)
style.configure('Dark.TButton', background=BUTTON_COLOR, foreground=FOREGROUND_COLOR)

tab_parent = ttk.Notebook(root)
create_tab(tab_parent, "SVM", svm_model)
create_tab(tab_parent, "ANN", ann_model)
tab_parent.pack(expand=1, fill='both')

root.mainloop()